In [1]:
from app.vectordb import build_database
%load_ext autoreload
%autoreload 2

In [2]:
build_database()

Database built successfully.


Collection(name=naics_collection-openai)

In [3]:
from app.vectordb import load_database

naics_collection = load_database()

In [46]:
from app.prediction import Prediction
%load_ext autoreload
%autoreload 2

pred = Prediction(naics_collection,n_results=1)

The autoreload extension is already loaded. To reload it, use:
  %reload_ext autoreload


Fetching 4 files: 100%|██████████| 4/4 [00:00<00:00, 27458.62it/s]


In [47]:
question = "How many software development companies are there in Silicon Valley?"

qemb = pred.pipeline(question)

Asking to truncate to max_length but no maximum length is provided and the model has no predefined maximum length. Default to no truncation.


In [48]:
qemb

(['Professional, Scientific, and Technical Services'],
 ['54'],
 ['Custom Computer Programming Services'],
 ['541511'])

In [17]:
question_emb = pred.emb_fn([question])

In [4]:
from app.gliner import get_entities

entities = get_entities(pred.gliner_model,question)

In [5]:
query_texts = [entity["text"] for entity in entities]
query_texts

['software development companies']

In [18]:
rdocs = pred.query_database(question_emb,"parent")

In [20]:
parent_names = [parent_name for parent_name in rdocs['documents'][0]]
parent_names

['Professional, Scientific, and Technical Services']

In [23]:
relevant_docs = pred.naics_collection.query(
            query_embeddings=question_emb,
            # where={
            #     "$and": [
            #         {"TYPE": {"$eq": "CHILD"}},
            #         {"metadata_field": {"$eq": parent_names[0]}},
            #     ]
            # },
            where={"TYPE": "PARENT"},
            n_results=3,
        )

In [30]:
from app.vectordb import _get_docs_metadata
%load_ext autoreload
%autoreload 2
docs,metadata = _get_docs_metadata()

The autoreload extension is already loaded. To reload it, use:
  %reload_ext autoreload


In [31]:
docs[1],metadata[1]

('Industries in the Crop Production subsector grow crops mainly for food and fiber.  The subsector comprises establishments, such as farms, orchards, groves, greenhouses, and nurseries, primarily engaged in growing crops, plants, vines, or trees and their seeds.The industries in this subsector are grouped by similarity of production activity, including biological and physiological characteristics and economic requirements, the length of growing season, degree of crop rotation, extent of input specialization, labor requirements, and capital demands.  The production process is typically completed when the raw product or commodity grown reaches the "farm gate" for market, that is, at the point of first sale or price determination.Establishments are classified in the Crop Production subsector when crop production (i.e., value of crops for market) accounts for one-half or more of the establishment\'s total agricultural production.  Within the subsector, establishments are classified in a s'

In [24]:
parent_names = [parent_name for parent_name in relevant_docs['documents'][0]]
parent_names

['Professional, Scientific, and Technical Services',
 'Management of Companies and Enterprises',
 'Manufacturing']

In [8]:
parent_names = [parent_name[0] for parent_name in rdocs['documents']]
parent_names

['Professional, Scientific, and Technical Services']

In [7]:
# rdocs['embeddings'][0][0]

In [16]:
child_docs = pred.query_database(rdocs['embeddings'][0],"child",parent_names)

In [15]:
relevant_docs = pred.naics_collection.query(
            query_embeddings=qemb,
            # where={
            #     "$and": [
            #         {"TYPE": {"$eq": "CHILD"}},
            #         {"metadata_field": {"$eq": parent_names[0]}},
            #     ]
            # },
            where={"TYPE": "CHILD"},
            n_results=1,
        )

In [16]:
relevant_docs

{'ids': [['956']],
 'distances': [[0.7416042685508728]],
 'metadatas': [[{'DESCRIPTION': ' See industry description for 513210. This industry comprises establishments primarily engaged in software publishing.  Establishments in this industry carry out operations necessary for producing and distributing computer software, such as designing, providing documentation, assisting in installation, and providing support services to software purchasers.  These establishments may design, develop, and publish, or publish only.  These establishments may publish and distribute software through subscriptions and/or downloads. Cross-References. Establishments primarily engaged in--',
    'NAICS CODE': '5132, 51321, 513210',
    'PARENT': 'Information',
    'TYPE': 'CHILD'}]],
 'embeddings': None,
 'documents': [['Software Publishers']],
 'uris': None,
 'data': None}